<a href="https://colab.research.google.com/github/sunshineluyao/ESG_TwoSources/blob/main/Sorting/Sorting_Method_Abnormal_Return_MSCI_2020_2_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Creat the Leader, Average, and Laggard Group

In [2]:
import pandas as pd
import numpy as np

In [4]:
df =pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/MSCI_ESG_Table_With_SepaScore.csv')
df.head()

,Ticker,Month,Company_name,Stock_exchange,Industry,SIC_Code,SIC_2,BV_Y1,BV_Y2,BV_Y3,BV_Y4,BV_Y5,Longterm_BV,MV_Y1,MV_Y2,MV_Y3,MV_Y4,MV_Y5,Longterm_MV,Longterm_BMR,Industry_Ave_BMR,BookValue_Quarterly,MarketValue_Quarterly,BMR_Quarterly,Industry_adjusted_BMR_Quarterly,Size,Momentum,ESG_Rating,ESG_Score,Size_G,BMR_G,Mom_G,ESG_G,Monthly_Return,Porfolio_Return,ESG_Portfolio_Return,ESG_Abnormal_Return,Name,ESG_Overall_Rating,ESG_Overall_Score,E_Score,S_Score,G_Score,E_G,S_G,G_G
0,A,1,Agilent Technologies Inc,NYSE,Healthcare,3826,38,4243.000,4831.000,4567.000,4748.000,4873.000,4652.400,13888.064911,14658.833458,21632.517268,21488.239823,2.646175e+04,19625.880519,0.237054,0.307958,4848.000,2.646175e+04,0.183208,-0.124750,26461.710,11.63787,AAA,8.8,5,3,2,Leader_Group,-3.223538,-2.903777,-5.848175,-2.944398,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9,E_Leader,S_Learder,G_Average
1,AAL,1,American Airlines Group Inc,NASDAQ,Industrials,4512,45,5635.000,3785.000,-780.000,-169.000,-118.000,1670.600,26694.286577,24191.263534,24896.306768,14789.270777,1.256350e+04,20626.926053,0.080991,0.618105,-118.000,1.256350e+04,-0.009392,-0.627498,12280.780,-28.43626,CCC,0.7,4,1,1,Laggard_Group,-6.415624,-3.839508,-8.223340,-4.383832,"American Airlines Group, Inc.",CCC,0.7,3.3,2.5,5.7,E_Average,S_Laggard,G_Average
2,AAON,1,Aaon Inc,NASDAQ,Industrials,3585,35,178.918,205.898,237.226,249.443,290.140,232.325,1256.035791,1743.560682,1925.589326,1826.434467,2.573199e+03,1864.963938,0.124573,0.360987,290.140,2.573199e+03,0.112755,-0.248233,2574.162,30.09222,A,5.8,2,1,4,Average_Group,6.132364,-0.441837,1.451676,1.893513,"AAON, Inc.",A,5.8,5.5,8.4,5.7,E_Average,S_Learder,G_Average
3,AAP,1,Advance Auto Parts Inc,NYSE,Consumer Cyclicals,5531,55,2460.648,2916.192,3415.196,3550.813,3549.081,3178.386,11022.840214,12456.301060,7366.895907,11477.844994,1.109258e+04,10683.293408,0.297510,0.375682,3549.081,1.109258e+04,0.319951,-0.055731,11092.590,-11.60880,BBB,4.6,4,3,1,Average_Group,-17.738520,-4.628674,-9.200801,-4.572127,"Advance Auto Parts, Inc.",BBB,4.6,2.0,3.8,6.6,E_Laggrad,S_Average,G_Average
4,AAPL,1,Apple Inc,NASDAQ,Technology,3663,36,128249.000,134047.000,107147.000,90488.000,65339.000,105054.000,583612.701620,608960.249120,868879.619760,748539.080520,1.304765e+06,822951.283654,0.127655,0.395727,89531.000,1.304765e+06,0.068618,-0.327109,1304765.000,49.65282,A,6.4,5,1,5,Average_Group,5.400991,1.921124,3.635589,1.714465,Apple Inc.,A,6.4,3.6,5.8,4.8,E_Average,S_Learder,G_Average


In [6]:
df =df[['Ticker',	'Month',	'Size',	'Momentum', 'Size_G',	'BMR_G',	'Mom_G','Monthly_Return','Porfolio_Return']]
df.head()

,Ticker,Month,Size,Momentum,Size_G,BMR_G,Mom_G,Monthly_Return,Porfolio_Return
0,A,1,26461.710,11.63787,5,3,2,-3.223538,-2.903777
1,AAL,1,12280.780,-28.43626,4,1,1,-6.415624,-3.839508
2,AAON,1,2574.162,30.09222,2,1,4,6.132364,-0.441837
3,AAP,1,11092.590,-11.60880,4,3,1,-17.738520,-4.628674
4,AAPL,1,1304765.000,49.65282,5,1,5,5.400991,1.921124


In [8]:
df['Abnormal_Return']=df['Monthly_Return']-df['Porfolio_Return']
df.head()

,Ticker,Month,Size,Momentum,Size_G,BMR_G,Mom_G,Monthly_Return,Porfolio_Return,Abnormal_Return
0,A,1,26461.710,11.63787,5,3,2,-3.223538,-2.903777,-0.319761
1,AAL,1,12280.780,-28.43626,4,1,1,-6.415624,-3.839508,-2.576116
2,AAON,1,2574.162,30.09222,2,1,4,6.132364,-0.441837,6.574201
3,AAP,1,11092.590,-11.60880,4,3,1,-17.738520,-4.628674,-13.109846
4,AAPL,1,1304765.000,49.65282,5,1,5,5.400991,1.921124,3.479867


In [10]:
ESG = pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/Factor%20Model/ESG_score_2020Jan.csv')
ESG.head()

,Symbol,Name,MSCI ESG Rtg Overall Company Rating,MSCI ESG Rtg Overall Industry-Adjusted Company Score,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Social,MSCI ESG Rtg Score Governance
0,A,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9
1,AAL,"American Airlines Group, Inc.",CCC,0.7,3.3,2.5,5.7
2,AAN,Aaron's Company Inc,@NA,NaN,NaN,NaN,NaN
3,AAOI,"Applied Optoelectronics, Inc.",@NA,NaN,NaN,NaN,NaN
4,AAON,"AAON, Inc.",A,5.8,5.5,8.4,5.7


In [11]:
ESG.isna().sum()

Symbol                                                   0
Name                                                     0
MSCI ESG Rtg Overall Company Rating                      0
MSCI ESG Rtg Overall Industry-Adjusted Company Score    32
MSCI ESG Rtg Score Environmental                        32
MSCI ESG Rtg Score Social                               32
MSCI ESG Rtg Score Governance                           32
dtype: int64

In [12]:
ESG=ESG.dropna()

In [13]:
ESG['ESG_G']=pd.qcut(ESG['MSCI ESG Rtg Overall Industry-Adjusted Company Score'],5,labels=np.arange(1,6,1))
ESG['E_G']=pd.qcut(ESG['MSCI ESG Rtg Score Environmental'],5,labels=np.arange(1,6,1))
ESG['S_G']=pd.qcut(ESG['MSCI ESG Rtg Score Social'],5,labels=np.arange(1,6,1))
ESG['G_G']=pd.qcut(ESG['MSCI ESG Rtg Score Governance'],5,labels=np.arange(1,6,1))

In [14]:
ESG['ESG_Group']=ESG['ESG_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
ESG['E_Group']=ESG['E_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
ESG['S_Group']=ESG['S_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
ESG['G_Group']=ESG['G_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')

In [16]:
df=df.merge(ESG, left_on='Ticker', right_on='Symbol')
df.head()

,Ticker,Month,Size,Momentum,Size_G,BMR_G,Mom_G,Monthly_Return,Porfolio_Return,Abnormal_Return,Symbol,Name,MSCI ESG Rtg Overall Company Rating,MSCI ESG Rtg Overall Industry-Adjusted Company Score,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Social,MSCI ESG Rtg Score Governance,ESG_G,E_G,S_G,G_G,ESG_Group,E_Group,S_Group,G_Group
0,A,1,26461.71,11.637870,5,3,2,-3.223538,-2.903777,-0.319761,A,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9,5,5,5,3,Leader,Leader,Leader,Average
1,A,2,25608.71,26.460120,5,3,2,-6.649709,-7.847952,1.198243,A,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9,5,5,5,3,Leader,Leader,Leader,Average
2,A,3,23950.89,8.574427,5,3,2,-7.071489,-12.705259,5.633770,A,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9,5,5,5,3,Leader,Leader,Leader,Average
3,A,4,22502.34,-5.132937,5,3,1,11.230420,8.530156,2.700264,A,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9,5,5,5,3,Leader,Leader,Leader,Average
4,A,5,23737.85,-12.187340,5,3,2,18.771050,8.965679,9.805371,A,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9,5,5,5,3,Leader,Leader,Leader,Average


# ESG: Visualizing Abnormal Return Monthly

In [17]:
import plotly.express as px
fig = px.box(df, x="ESG_Group", y="Abnormal_Return",color='ESG_Group',animation_frame='Month',animation_group='Symbol')
fig.show()

In [18]:
ESG_summary = df.groupby(by=['Month','ESG_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
ESG_summary['sqrt_count']=ESG_summary['count'].apply(lambda x: np.sqrt(x))
ESG_summary['mean_std']=ESG_summary['std']/ESG_summary['sqrt_count']
ESG_summary.head()

,Month,ESG_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,-0.346361,8.122034,860,29.325757,0.276959
1,1,Laggard,0.153909,9.305591,283,16.822604,0.553160
2,1,Leader,0.886666,7.147001,260,16.124515,0.443238
3,2,Average,-0.308584,8.262186,860,29.325757,0.281738
4,2,Laggard,1.102197,10.543337,283,16.822604,0.626736


In [19]:
import plotly.express as px
fig = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig.show()

In [20]:
ESG_summary = ESG_summary[ESG_summary['ESG_Group'].isin(['Leader', 'Laggard'])]

In [21]:
import plotly.express as px
fig = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig.show()

### E result

In [22]:
E_summary = df.groupby(by=['Month','E_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
E_summary['sqrt_count']=ESG_summary['count'].apply(lambda x: np.sqrt(x))
E_summary['mean_std']=ESG_summary['std']/ESG_summary['sqrt_count']
E_summary.head()

,Month,E_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,0.693953,8.326122,890,NaN,NaN
1,1,Laggard,-3.181992,8.136747,275,16.822604,0.553160
2,1,Leader,0.981716,6.940721,238,16.124515,0.443238
3,2,Average,-0.057410,8.988702,890,NaN,NaN
4,2,Laggard,-1.335331,7.447800,275,16.822604,0.626736


In [23]:
E_summary = E_summary[E_summary['E_Group'].isin(['Leader', 'Laggard'])]

In [24]:
import plotly.express as px
fig = px.line(E_summary, x="Month", y="mean", color="E_Group", error_y="mean_std")
fig.show()

### S result

In [25]:
S_summary = df.groupby(by=['Month','S_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
S_summary['sqrt_count']=S_summary['count'].apply(lambda x: np.sqrt(x))
S_summary['mean_std']=S_summary['std']/S_summary['sqrt_count']
S_summary.head()

,Month,S_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,-0.233733,8.059984,859,29.308702,0.275003
1,1,Laggard,0.704640,8.407618,284,16.852300,0.498900
2,1,Leader,-0.088931,8.496237,260,16.124515,0.526914
3,2,Average,-0.259516,8.817657,859,29.308702,0.300855
4,2,Laggard,0.790217,8.699664,284,16.852300,0.516230


In [26]:
S_summary = S_summary[S_summary['S_Group'].isin(['Leader', 'Laggard'])]

In [27]:
import plotly.express as px
fig = px.line(S_summary, x="Month", y="mean", color="S_Group", error_y="mean_std")
fig.show()

### G result

In [28]:
G_summary = df.groupby(by=['Month','G_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
G_summary['sqrt_count']=G_summary['count'].apply(lambda x: np.sqrt(x))
G_summary['mean_std']=G_summary['std']/G_summary['sqrt_count']
G_summary.head()

,Month,G_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,-0.151824,8.110122,849,29.137605,0.278339
1,1,Laggard,0.875074,9.064321,306,17.492856,0.518173
2,1,Leader,-0.655869,7.374611,248,15.748016,0.468288
3,2,Average,-0.128525,8.522106,849,29.137605,0.292478
4,2,Laggard,0.651449,10.308110,306,17.492856,0.589275


In [29]:
G_summary = G_summary[G_summary['G_Group'].isin(['Leader', 'Laggard'])]

In [30]:
import plotly.express as px
fig = px.line(G_summary, x="Month", y="mean", color="G_Group", error_y="mean_std")
fig.show()

# Step 3: Conduct Two-group t-test for each month

In [31]:
pip install pingouin

     |████████████████████████████████| 225kB 4.1MB/s 
  Created wheel for pingouin: filename=pingouin-0.3.9-cp36-none-any.whl size=223375 sha256=6a8b18d8651abce35f19f598614e9bba071bc38d6b483af3c27f4a8245f5a090
  Stored in directory: /root/.cache/pip/wheels/6c/d7/26/a23b9ed74061adc5cf8f5e016bbe2902a8ebf63b6471047dbc
  Created wheel for outdated: filename=outdated-0.2.0-cp36-none-any.whl size=4961 sha256=ecdc57df097d56c3f0e1d4a92ac236517ccaae0b499cfbd7d5cfa86c851dbdeb
  Stored in directory: /root/.cache/pip/wheels/fd/7c/ef/814f514d31197310872b5abf353feb8fef9d67ee658e1e7e39
  Created wheel for littleutils: filename=littleutils-0.2.2-cp36-none-any.whl size=7051 sha256=bdaa8727be6e0bc8ac77ee84262f53fcb737d67d0a31895c63db745764fa315b
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built pingouin outdated littleutils


In [32]:
import pingouin as pg


Two-Sample T-Test

https://pingouin-stats.org/generated/pingouin.ttest.html#pingouin.ttest

### Test Monthly the Leaders' Abornormal Return >0?

In [33]:
Leader = df[(df['Month']==1) & (df['ESG_Group']=='Leader')]['Abnormal_Return']
t_test = pg.ttest(Leader, 0)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,2.000428,259,two-sided,0.046498,"[0.01, 1.76]",0.124061,0.495,0.513219


In [34]:
df_ = pd.DataFrame()
Month_List =df.Month.unique().tolist()
for i in Month_List:
  Leader =  df[(df['Month']==i) & (df['ESG_Group']=='Leader')]['Abnormal_Return']
  t_test = pg.ttest(Leader, 0)
  t_test['Month']=i
  df_=df_.append(t_test)

In [35]:
df_

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power,Month
T-test,2.000428,259,two-sided,0.046498,"[0.01, 1.76]",0.124061,0.495,0.513219,1
T-test,-1.076578,259,two-sided,0.282671,"[-1.4, 0.41]",0.066767,0.123,0.188651,2
T-test,0.736680,259,two-sided,0.461984,"[-1.23, 2.71]",0.045687,0.091,0.113628,3
T-test,-1.215772,259,two-sided,0.225179,"[-3.16, 0.75]",0.075399,0.144,0.227779,4
T-test,0.185770,259,two-sided,0.852771,"[-1.09, 1.31]",0.011521,0.071,0.053933,5
T-test,1.101284,259,two-sided,0.271796,"[-0.44, 1.56]",0.068299,0.126,0.195252,6
T-test,2.250372,259,two-sided,0.025265,"[0.2, 2.99]",0.139562,0.83,0.611064,7
T-test,1.328756,259,two-sided,0.185098,"[-0.4, 2.04]",0.082406,0.166,0.262857,8
T-test,0.467519,259,two-sided,0.640522,"[-0.76, 1.24]",0.028994,0.077,0.075203,9
T-test,1.456606,259,two-sided,0.146436,"[-0.29, 1.92]",0.090335,0.197,0.305785,10


In [36]:
df_2 = pd.DataFrame()
Month_List =df.Month.unique().tolist()
for i in Month_List:
  Leader = df[(df['Month']==i) & (df['ESG_Group']=='Leader')]['Abnormal_Return']
  Laggard = df[(df['Month']==i) & (df['ESG_Group']=='Laggard')]['Abnormal_Return']
  t_test = pg.ttest(Leader, Laggard,paired=False)
  t_test['Month']=i
  df_2=df_2.append(t_test)

In [37]:
df_2

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power,Month
T-test,1.033749,524.811670,two-sided,0.301729,"[-0.66, 2.13]",0.087838,0.161,0.175231,1
T-test,-2.054846,507.084085,two-sided,0.040406,"[-3.12, -0.07]",0.174001,0.742,0.524729,2
T-test,0.550542,540.989072,two-sided,0.582175,"[-2.05, 3.64]",0.047116,0.111,0.084985,3
T-test,-0.041252,538.285288,two-sided,0.967110,"[-2.99, 2.87]",0.003521,0.096,0.050192,4
T-test,0.096033,540.033545,two-sided,0.923530,"[-1.61, 1.78]",0.008235,0.096,0.051050,5
T-test,1.079282,540.340311,two-sided,0.280944,"[-0.66, 2.28]",0.092247,0.168,0.188474,6
T-test,2.165176,519.047463,two-sided,0.030829,"[0.19, 3.84]",0.186961,0.93,0.584173,7
T-test,2.267872,518.274273,two-sided,0.023748,"[0.25, 3.43]",0.195860,1.159,0.623988,8
T-test,1.065951,530.545172,two-sided,0.286930,"[-0.62, 2.08]",0.091787,0.166,0.187061,9
T-test,2.740266,537.593203,two-sided,0.006343,"[0.66, 3.99]",0.233775,3.628,0.775331,10


# Overall test

In [52]:
Leader =df[df['ESG_Group']=='Leader']['Abnormal_Return']
Laggard=df[df['ESG_Group']=='Laggard']['Abnormal_Return']

In [53]:
t_test = pg.ttest(Leader, Laggard, correction='auto', paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,2.766746,6511.591194,two-sided,0.005678,"[0.22, 1.31]",0.068311,1.27,0.786477


In [54]:
E_Leader =df[df['E_Group']=='Leader']['Abnormal_Return']
E_Laggard=df[df['E_Group']=='Laggard']['Abnormal_Return']

In [55]:
t_test = pg.ttest(E_Leader, E_Laggard,correction='auto', paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,3.222217,6146.715897,two-sided,0.001279,"[0.35, 1.43]",0.081305,5.09,0.888923


In [56]:
S_Leader =df[df['S_Group']=='Leader']['Abnormal_Return']
S_Laggard=df[df['S_Group']=='Laggard']['Abnormal_Return']

In [57]:
t_test = pg.ttest(S_Leader, S_Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,2.004587,6419.565948,two-sided,0.045049,"[0.01, 1.1]",0.049758,0.207,0.519145


In [58]:
G_Leader =df[df['G_Group']=='Leader']['Abnormal_Return']
G_Laggard=df[df['G_Group']=='Laggard']['Abnormal_Return']

In [59]:
t_test = pg.ttest(G_Leader, G_Laggard,correction='auto', paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,-1.197189,6644.365028,two-sided,0.231275,"[-0.86, 0.21]",0.028941,0.057,0.21657


In [60]:
df.to_csv("ESG_MSCI_FINAL.csv")